# S-P Visualization from SC3 Phase Picks

**Given the location scatter for earthquakes near White Island, the volcano monitoring meeting wondered about using S-P interval to see if any hypocentral 'migration' can be seen. Previous efforts to look at chnages in S-P at volcanoes, such as Raoul Island in 2006' has relied on manually picking P- and S-phases. In my view, manual picking is no longer the best use of my time. Consequently, this notebook uses picks routinely made using earthquake location with SC3. PIcks are extracted from GeoNet's FDSN client using the get_events function**

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client as FDSN_Client
from obspy import read_inventory
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
client = FDSN_Client("GEONET")

## Query FDSN web server to get event list

**This uses a simple distance-based search, other options are possible**

In [ ]:
starttime = UTCDateTime('2019-05-22 00:00:00.000')
endtime = UTCDateTime.now()
latitude = -37.521
longitude = 177.184
maxradius = 0.20
maxdepth = 30

sites = ['WSRZ', 'WIZ']

In [ ]:
cat = client.get_events(starttime=starttime, endtime=endtime, latitude=latitude, longitude=longitude, maxradius=maxradius, maxdepth=maxdepth)

In [ ]:
dfp = pd.DataFrame()
dfs = pd.DataFrame()
df = pd.DataFrame()

### Extract the required data. Loop is messy, but based on data tutorial. A better method?

Write to Pandas dataframes, one for P- and one for S-phases. Makes subsequent data management easier.

In [ ]:
for ev in cat:
    ot = ev.short_str().split('|')[0] # better way to do this?
    origin = ev.origins[0]
    for p in range(len(ev.picks)):
        for a in range(len(origin.arrivals)):
            if ev.picks[p].resource_id == origin.arrivals[a].pick_id:
                for site in sites:
                    if ev.picks[p].waveform_id['station_code'] == site:
                        vals = {'origin_time':ot, 'site':ev.picks[p].waveform_id['station_code'], 'phase':origin.arrivals[a].phase, 'pick':ev.picks[p].time}
                        data = pd.DataFrame([vals], columns=vals.keys())
                        if (origin.arrivals[a].phase == 'P'):
                            dfp = dfp.append(data, ignore_index=False)
                        else:
                            dfs = dfs.append(data, ignore_index=False)

### Merge P and S dataframes and prepare for visualization

In [ ]:
df = pd.merge(dfp, dfs, how='outer', on=['origin_time', 'site'])
df['origin_time'] = df['origin_time'].apply(pd.to_datetime)
df.rename(columns={'phase_x':'P-phase', 'phase_y':'S-phase'}, inplace=True)
# df.set_index(keys='origin_time', inplace=True)
df['sminusp'] = df['pick_y'] - df['pick_x']
df = df.drop(columns=['pick_x', 'pick_y'])
df['sminusp'] = df['sminusp'].astype('float', )

In [ ]:
df.head()

## Data summary

### How much S-P data

In [ ]:
num_pphases_wiz = len(df[(df['site'] == 'WIZ') & (df['P-phase'] == 'P')])
num_pphases_wsrz = len(df[(df['site'] == 'WSRZ') & (df['P-phase'] == 'P')])
pd.notna(df['sminusp'])
num_sp_wiz = len(df[(df['site'] == 'WIZ') & pd.notna(df['sminusp'])])
num_sp_wsrz = len(df[(df['site'] == 'WSRZ') & pd.notna(df['sminusp'])])

print ('P-phases WIZ =', num_pphases_wiz)
print ('P-phases WSRZ =', num_pphases_wsrz)
print ('S-P WIZ =', num_sp_wiz)
print ('S-P WSRZ =', num_sp_wsrz)

## Visualization

### S-P vs time

In [ ]:
wiz = df[df['site']=='WIZ']
wsrz = df[df['site']=='WSRZ']
scat = wiz.plot(x='origin_time', y='sminusp', marker='o', linestyle='None', title='S-P Interval', figsize=(10,5), label='WIZ')
wsrz.plot(ax=scat, x='origin_time', y='sminusp', marker='o', linestyle='None', color='red', label='WSRZ')
scat.legend(loc='best')
scat.set_ylabel('S-P (s)')
scat.set_ylim(top=4)

fig = scat.get_figure()
fig.savefig('white_island_s-p_scatter.png', dpi=100)

### Boxplot

In [ ]:
bp = df.boxplot(column='sminusp', by='site', whis=[5,95])
bp.set_title('S-P by site (5-95 whiskers), with outliers')
bp.set_ylabel('S-P (s)')
plt.suptitle('')

fig = bp.get_figure()
fig.savefig('white_island_s-p_boxplot.png', dpi=100)